In [ ]:
!pip install transformers accelerate

In [22]:
import pandas as pd

df = pd.read_csv("/kaggle/input/amazon-train-with-ocr/test_0_to_66000_with_ocr.csv")
df['extracted_texts'] = df['extracted_texts'].apply(lambda x: eval(x))
small_df = df

66000

In [6]:
import transformers
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

model_id = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
cnt = 0
def get_llama_output(extracted_texts, entity_name):
    global cnt
    print(cnt)
    cnt += 1
    if len(extracted_texts) == 0:
        return ""
    
    entity_unit_map = {
      "width": ["centimetre", "foot", "millimetre", "metre", "inch", "yard"],
      "depth": ["centimetre", "foot", "millimetre", "metre", "inch", "yard"],
      "height": ["centimetre", "foot", "millimetre", "metre", "inch", "yard"],
      "item_weight": ["milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"],
      "maximum_weight_recommendation": ["milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"],
      "voltage": ["millivolt", "kilovolt", "volt"],
      "wattage": ["kilowatt", "watt"],
      "item_volume": ["cubic foot", "microlitre", "cup", "fluid ounce", "centilitre", "imperial gallon", "pint", "decilitre", "litre", "millilitre", "quart", "cubic inch", "gallon"]
    }

    messages = [
        {"role": "system", "content": "You are a measurement extractor. You are given pieces of broken information \
                                        about an object. You are also given a list of allowed units. Your job is to answer ONLY with the value asked followed by the unit mentioned \
                                        in the information. \
                                        If no unit is mentioned in the information, choose an appropriate unit from the list of allowed units. \
                                        Answer in the format \"val unit\". Do not give any reasoning."},
        {"role": "user", "content": "Who are you?"},
    ]

    input_template = """Information: {info}
                        Allowed units: {units}

                        What is the {entity_name}"""
    units = entity_unit_map[entity_name]
    messages[1]["content"] = input_template.format(info=extracted_texts, entity_name=entity_name, units = units)
    outputs = pipeline(
    messages,
    max_new_tokens=256,
    )
    return outputs[0]["generated_text"][-1]["content"]
    

In [ ]:
small_df['llama_output'] = small_df.apply(lambda x: get_llama_output(x['extracted_texts'], x['entity_name']), axis=1)

In [ ]:
small_df.to_csv("test_0_to_66000_llama.csv")